In [ ]:
import dataclasses as dc
import pathlib
import random
import uuid

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import pendulum
import seaborn as sns

In [ ]:
from orchid import (project_loader as opl)

In [ ]:
import toolz.curried as toolz

In [ ]:
# noinspection PyUnresolvedReferences
import UnitsNet

In [ ]:
project_filenames = {
    'bakken': 'frankNstein_Bakken_UTM13_FEET.ifrac',
    'montney': 'Project-frankNstein_Montney_UTM13_METERS.ifrac',
}

In [ ]:
test_data_path = pathlib.Path('c:/src/Orchid.IntegrationTestData/')
project_path_names = toolz.valmap(lambda fn: test_data_path.joinpath(fn), project_filenames)
project_path_names

In [ ]:
projects = toolz.valmap(
    lambda pn: opl.ProjectLoader(str(pn)).native_project(),
    project_path_names)
projects

In [ ]:
def get_project_monitors(proj):
    return proj.Monitors.Items

project_monitors = toolz.pipe(
    projects,
    toolz.valmap(get_project_monitors),
)
project_monitors

In [ ]:
toolz.pipe(project_monitors.values(),
           toolz.map(len),
           list,)

In [ ]:
@dc.dataclass
class MonitorBasics:
    object_id: uuid.UUID
    name: str
    display_name: str
    start_time: pendulum.DateTime
    stop_time: pendulum.DateTime

@toolz.curry
def summarize_monitor_basics(m):
    result = MonitorBasics(m.ObjectId, m.DisplayName, m.Name, m.StartTime.ToString('o'), m.StopTime.ToString('o'))
    return result

project_monitor_summaries = toolz.pipe(
    project_monitors,
    toolz.valmap(toolz.map(summarize_monitor_basics)),
    toolz.valmap(list),
)
project_monitor_summaries

In [ ]:
selected_field_name = 'bakken'

In [1]:
def make_id_summary(field_name, summaries):
    field_name_summaries = summaries[field_name]
    return {
        'field': field_name,
        'Object Id': [s.object_id for s in field_name_summaries],
        # 'Display Name': [s.display_name for s in field_name_summaries],
        # 'Name': [s.name for s in field_name_summaries],
        'Start Time': [s.start_time for s in field_name_summaries],
        'Stop Time': [s.stop_time for s in field_name_summaries],
    }

In [ ]:
pd.DataFrame(data=make_id_summary('bakken', project_monitor_summaries))

In [ ]:
pd.DataFrame(data=make_id_summary('montney', project_monitor_summaries))

In [ ]:
def merge_time_series(so_far, m):
    return toolz.assoc(so_far, m.DisplayName, m)

project_monitor_time_series = toolz.pipe(
    project_monitors,
    toolz.valmap(toolz.map(lambda m: m.TimeSeries)),
    toolz.valmap(list),
    # toolz.valmap(toolz.reduce(merge_time_series)),
)
project_monitor_time_series